In [1]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False
import gc


# 인코더 추가
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
#VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor
#상수항추가
from statsmodels.tools.tools import add_constant
# 카이제곱, ANOVA
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway
#Turkeyhsd
from statsmodels.stats.multicomp import pairwise_tukeyhsd

### 데이터 불러오기

In [3]:
# parquet 파일 데이터를 읽어온다.
# x 값으로 쓸 데이터 
df1_train = pd.read_parquet('open/train/3.승인매출정보/201807_train_승인매출정보.parquet')
df2_train = pd.read_parquet('open/train/3.승인매출정보/201808_train_승인매출정보.parquet')
df3_train = pd.read_parquet('open/train/3.승인매출정보/201809_train_승인매출정보.parquet')
df4_train = pd.read_parquet('open/train/3.승인매출정보/201810_train_승인매출정보.parquet')
df5_train = pd.read_parquet('open/train/3.승인매출정보/201811_train_승인매출정보.parquet')
df6_train = pd.read_parquet('open/train/3.승인매출정보/201812_train_승인매출정보.parquet')

In [4]:
# 결과데이터를 불러온다. 
segment_df = pd.read_csv('Segment.csv')
segment_df

,기준년월,ID,Segment
0,201807,TRAIN_000000,D
1,201807,TRAIN_000001,E
2,201807,TRAIN_000002,C
3,201807,TRAIN_000003,D
4,201807,TRAIN_000004,E
...,...,...,...
2399995,201812,TRAIN_399995,E
2399996,201812,TRAIN_399996,D
2399997,201812,TRAIN_399997,C
2399998,201812,TRAIN_399998,E


In [5]:
# 데이터프레임의 컬럼 이름을 리스트에 담는다. 
column_list = df1_train.columns.tolist()

In [7]:
print(column_list)

['기준년월', 'ID', '최종이용일자_기본', '최종이용일자_신판', '최종이용일자_CA', '최종이용일자_카드론', '최종이용일자_체크', '최종이용일자_일시불', '최종이용일자_할부', '이용건수_신용_B0M', '이용건수_신판_B0M', '이용건수_일시불_B0M', '이용건수_할부_B0M', '이용건수_할부_유이자_B0M', '이용건수_할부_무이자_B0M', '이용건수_부분무이자_B0M', '이용건수_CA_B0M', '이용건수_체크_B0M', '이용건수_카드론_B0M', '이용금액_일시불_B0M', '이용금액_할부_B0M', '이용금액_할부_유이자_B0M', '이용금액_할부_무이자_B0M', '이용금액_부분무이자_B0M', '이용금액_CA_B0M', '이용금액_체크_B0M', '이용금액_카드론_B0M', '이용후경과월_신용', '이용후경과월_신판', '이용후경과월_일시불', '이용후경과월_할부', '이용후경과월_할부_유이자', '이용후경과월_할부_무이자', '이용후경과월_부분무이자', '이용후경과월_CA', '이용후경과월_체크', '이용후경과월_카드론', '이용건수_신용_R12M', '이용건수_신판_R12M', '이용건수_일시불_R12M', '이용건수_할부_R12M', '이용건수_할부_유이자_R12M', '이용건수_할부_무이자_R12M', '이용건수_부분무이자_R12M', '이용건수_CA_R12M', '이용건수_체크_R12M', '이용건수_카드론_R12M', '이용금액_일시불_R12M', '이용금액_할부_R12M', '이용금액_할부_유이자_R12M', '이용금액_할부_무이자_R12M', '이용금액_부분무이자_R12M', '이용금액_CA_R12M', '이용금액_체크_R12M', '이용금액_카드론_R12M', '최대이용금액_일시불_R12M', '최대이용금액_할부_R12M', '최대이용금액_할부_유이자_R12M', '최대이용금액_할부_무이자_R12M', '최대이용금액_부분무이자_R12M', '최대이용금액_CA_R12M', '최대이용금액_체크_R12M', '

In [8]:
# 슬라이싱
column_list = column_list[223:309]

- 이용개월수_D페이_R6M까지 슬라이싱 되어야함

In [10]:
print(column_list)

['RP건수_B0M', 'RP금액_B0M', 'RP유형건수_B0M', 'RP건수_통신_B0M', 'RP건수_아파트_B0M', 'RP건수_제휴사서비스직접판매_B0M', 'RP건수_렌탈_B0M', 'RP건수_가스_B0M', 'RP건수_전기_B0M', 'RP건수_보험_B0M', 'RP건수_학습비_B0M', 'RP건수_유선방송_B0M', 'RP건수_건강_B0M', 'RP건수_교통_B0M', 'RP후경과월', 'RP후경과월_통신', 'RP후경과월_아파트', 'RP후경과월_제휴사서비스직접판매', 'RP후경과월_렌탈', 'RP후경과월_가스', 'RP후경과월_전기', 'RP후경과월_보험', 'RP후경과월_학습비', 'RP후경과월_유선방송', 'RP후경과월_건강', 'RP후경과월_교통', '증감_RP건수_전월', '증감_RP유형건수_전월', '증감_RP건수_통신_전월', '증감_RP건수_아파트_전월', '증감_RP건수_제휴사서비스직접판매_전월', '증감_RP건수_렌탈_전월', '증감_RP건수_가스_전월', '증감_RP건수_전기_전월', '증감_RP건수_보험_전월', '증감_RP건수_학습비_전월', '증감_RP건수_유선방송_전월', '증감_RP건수_건강_전월', '증감_RP건수_교통_전월', '최초카드론이용경과월', '최종카드론이용경과월', '카드론이용건수_누적', '카드론이용월수_누적', '카드론이용금액_누적', '최종카드론_금융상환방식코드', '최종카드론_신청경로코드', '최종카드론_대출이율', '최종카드론_대출일자', '최종카드론_대출월수', '최종카드론_거치개월수', '최종카드론_대출금액', '이용개월수_온라인_R6M', '이용개월수_오프라인_R6M', '이용금액_온라인_R6M', '이용금액_오프라인_R6M', '이용건수_온라인_R6M', '이용건수_오프라인_R6M', '이용금액_온라인_R3M', '이용금액_오프라인_R3M', '이용건수_온라인_R3M', '이용건수_오프라인_R3M', '이용금액_온라인_B0M', '이용금액_오프라인_B0M', '이용건수_온라인_B0M',

In [11]:
df1_train = df1_train[column_list]

In [12]:
df2_train = df2_train[column_list]
df3_train = df3_train[column_list]
df4_train = df4_train[column_list]
df5_train = df5_train[column_list]
df6_train = df6_train[column_list]

In [13]:
combined_df = pd.concat([df1_train, df2_train, df3_train, df4_train, df5_train, df6_train], 
                         axis=0,        # 행 방향
                         ignore_index=True)  # 인덱스 초기화

In [14]:
# 세그먼트와 데이터를 합쳐준다. 

all_df = pd.concat([segment_df,combined_df], axis = 1)
all_df

,기준년월,ID,Segment,RP건수_B0M,RP금액_B0M,RP유형건수_B0M,RP건수_통신_B0M,RP건수_아파트_B0M,RP건수_제휴사서비스직접판매_B0M,RP건수_렌탈_B0M,...,이용금액_페이_오프라인_B0M,이용건수_페이_온라인_B0M,이용건수_페이_오프라인_B0M,이용개월수_간편결제_R6M,이용개월수_당사페이_R6M,이용개월수_당사기타_R6M,이용개월수_A페이_R6M,이용개월수_B페이_R6M,이용개월수_C페이_R6M,이용개월수_D페이_R6M
0,201807,TRAIN_000000,D,2,147,2,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,201807,TRAIN_000001,E,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,201807,TRAIN_000002,C,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
3,201807,TRAIN_000003,D,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,201807,TRAIN_000004,E,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,E,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2399996,201812,TRAIN_399996,D,2,196,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2399997,201812,TRAIN_399997,C,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2399998,201812,TRAIN_399998,E,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
print(all_df.isna().sum().to_string())

기준년월                           0
ID                             0
Segment                        0
RP건수_B0M                       0
RP금액_B0M                       0
RP유형건수_B0M                     0
RP건수_통신_B0M                    0
RP건수_아파트_B0M                   0
RP건수_제휴사서비스직접판매_B0M            0
RP건수_렌탈_B0M                    0
RP건수_가스_B0M                    0
RP건수_전기_B0M                    0
RP건수_보험_B0M                    0
RP건수_학습비_B0M                   0
RP건수_유선방송_B0M                  0
RP건수_건강_B0M                    0
RP건수_교통_B0M                    0
RP후경과월                         0
RP후경과월_통신                      0
RP후경과월_아파트                     0
RP후경과월_제휴사서비스직접판매              0
RP후경과월_렌탈                      0
RP후경과월_가스                      0
RP후경과월_전기                      0
RP후경과월_보험                      0
RP후경과월_학습비                     0
RP후경과월_유선방송                    0
RP후경과월_건강                      0
RP후경과월_교통                      0
증감_RP건수_전월                     0
증감_RP유형건수_

In [16]:
for col in column_list:
    try:
        var_value = all_df[col].var()
        print(f"{col}: {var_value:.2f}")
    except TypeError:
        print(f"❌ 분산 계산 불가 (문자열 포함): {col}")


RP건수_B0M: 1.25
RP금액_B0M: 785702.14
RP유형건수_B0M: 0.71
RP건수_통신_B0M: 0.36
RP건수_아파트_B0M: 0.03
RP건수_제휴사서비스직접판매_B0M: 0.10
RP건수_렌탈_B0M: 0.00
RP건수_가스_B0M: 0.01
RP건수_전기_B0M: 0.00
RP건수_보험_B0M: 0.01
RP건수_학습비_B0M: 0.00
RP건수_유선방송_B0M: 0.00
RP건수_건강_B0M: 0.00
RP건수_교통_B0M: 0.55
RP후경과월: 8.64
RP후경과월_통신: 6.89
RP후경과월_아파트: 1.10
RP후경과월_제휴사서비스직접판매: 3.46
RP후경과월_렌탈: 0.18
RP후경과월_가스: 0.30
RP후경과월_전기: 0.14
RP후경과월_보험: 0.39
RP후경과월_학습비: 0.03
RP후경과월_유선방송: 0.00
RP후경과월_건강: 0.00
RP후경과월_교통: 6.08
증감_RP건수_전월: 0.08
증감_RP유형건수_전월: 0.02
증감_RP건수_통신_전월: 0.01
증감_RP건수_아파트_전월: 0.00
증감_RP건수_제휴사서비스직접판매_전월: 0.00
증감_RP건수_렌탈_전월: 0.00
증감_RP건수_가스_전월: 0.00
증감_RP건수_전기_전월: 0.00
증감_RP건수_보험_전월: 0.00
증감_RP건수_학습비_전월: 0.00
증감_RP건수_유선방송_전월: 0.00
증감_RP건수_건강_전월: 0.00
증감_RP건수_교통_전월: 0.05
최초카드론이용경과월: 133672.76
최종카드론이용경과월: 134678.57
카드론이용건수_누적: 3.80
카드론이용월수_누적: 2.74
카드론이용금액_누적: 7299260051.79
최종카드론_금융상환방식코드: 1.70
❌ 분산 계산 불가 (문자열 포함): 최종카드론_신청경로코드
최종카드론_대출이율: 35.65
최종카드론_대출일자: 170896323.05
최종카드론_대출월수: 84.71
최종카드론_거치개월수: 1.13
최종카드론_대출금액: 622323454.95
이용개월수_

In [17]:
def print_value_counts(df, columns=None, dropna=False):

    if columns is None:
        columns = df.columns.tolist()
    
    for col in columns:
        print(f"\n[🔢 Value Counts for '{col}']\n")
        print(df[col].value_counts(dropna=dropna))

In [18]:
zero_variance_cols  = []
# 각 컬럼별 분산 계산
for col in column_list:
    try:
        var_value = all_df[col].var()
        if var_value == 0:
            zero_variance_cols.append(col)
    except TypeError:
        # 숫자형이 아닌 경우 무시
        pass

# 결과 출력
print("📌 분산이 0인 컬럼:")
for col in zero_variance_cols:
    print(f"{col}")

📌 분산이 0인 컬럼:
RP건수_유선방송_B0M
RP건수_건강_B0M
RP후경과월_유선방송
RP후경과월_건강
증감_RP건수_유선방송_전월
증감_RP건수_건강_전월
이용개월수_당사페이_R6M


In [19]:
print_value_counts(all_df, columns=zero_variance_cols)


[🔢 Value Counts for 'RP건수_유선방송_B0M']

RP건수_유선방송_B0M
0    2400000
Name: count, dtype: int64

[🔢 Value Counts for 'RP건수_건강_B0M']

RP건수_건강_B0M
0    2400000
Name: count, dtype: int64

[🔢 Value Counts for 'RP후경과월_유선방송']

RP후경과월_유선방송
6    2400000
Name: count, dtype: int64

[🔢 Value Counts for 'RP후경과월_건강']

RP후경과월_건강
6    2400000
Name: count, dtype: int64

[🔢 Value Counts for '증감_RP건수_유선방송_전월']

증감_RP건수_유선방송_전월
0    2400000
Name: count, dtype: int64

[🔢 Value Counts for '증감_RP건수_건강_전월']

증감_RP건수_건강_전월
0    2400000
Name: count, dtype: int64

[🔢 Value Counts for '이용개월수_당사페이_R6M']

이용개월수_당사페이_R6M
0    2400000
Name: count, dtype: int64


- 값을 하나만 가진 데이터값은 상관관계 분석이나 추후 학습에 방해되므로 drop
- 빼야할 컬럼 []

In [ ]:
10/0

In [36]:
all_df.drop(['RP건수_유선방송_B0M','RP건수_건강_B0M','RP후경과월_유선방송',
'RP후경과월_건강','증감_RP건수_유선방송_전월','증감_RP건수_건강_전월','이용개월수_당사페이_R6M'], axis=1, inplace=True)

### 변수간 관계파악
Anova -> ETA제곱
카이제곱 -> 크레이머스V

In [38]:
print(column_list)

['RP건수_B0M', 'RP금액_B0M', 'RP유형건수_B0M', 'RP건수_통신_B0M', 'RP건수_아파트_B0M', 'RP건수_제휴사서비스직접판매_B0M', 'RP건수_렌탈_B0M', 'RP건수_가스_B0M', 'RP건수_전기_B0M', 'RP건수_보험_B0M', 'RP건수_학습비_B0M', 'RP건수_유선방송_B0M', 'RP건수_건강_B0M', 'RP건수_교통_B0M', 'RP후경과월', 'RP후경과월_통신', 'RP후경과월_아파트', 'RP후경과월_제휴사서비스직접판매', 'RP후경과월_렌탈', 'RP후경과월_가스', 'RP후경과월_전기', 'RP후경과월_보험', 'RP후경과월_학습비', 'RP후경과월_유선방송', 'RP후경과월_건강', 'RP후경과월_교통', '증감_RP건수_전월', '증감_RP유형건수_전월', '증감_RP건수_통신_전월', '증감_RP건수_아파트_전월', '증감_RP건수_제휴사서비스직접판매_전월', '증감_RP건수_렌탈_전월', '증감_RP건수_가스_전월', '증감_RP건수_전기_전월', '증감_RP건수_보험_전월', '증감_RP건수_학습비_전월', '증감_RP건수_유선방송_전월', '증감_RP건수_건강_전월', '증감_RP건수_교통_전월', '최초카드론이용경과월', '최종카드론이용경과월', '카드론이용건수_누적', '카드론이용월수_누적', '카드론이용금액_누적', '최종카드론_금융상환방식코드', '최종카드론_신청경로코드', '최종카드론_대출이율', '최종카드론_대출일자', '최종카드론_대출월수', '최종카드론_거치개월수', '최종카드론_대출금액', '이용개월수_온라인_R6M', '이용개월수_오프라인_R6M', '이용금액_온라인_R6M', '이용금액_오프라인_R6M', '이용건수_온라인_R6M', '이용건수_오프라인_R6M', '이용금액_온라인_R3M', '이용금액_오프라인_R3M', '이용건수_온라인_R3M', '이용건수_오프라인_R3M', '이용금액_온라인_B0M', '이용금액_오프라인_B0M', '이용건수_온라인_B0M',

In [40]:
# 크레이머스V와 eta를 구하기 위한 함수
def cramers_v(confusion_matrix):
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape
    return np.sqrt(chi2 / (n * (min(k, r) - 1)))

def eta_squared(anova_ss_between, total_ss):
    return anova_ss_between / total_ss if total_ss != 0 else np.nan

In [42]:
# 수치형/범주형 자동 구분
num_cols = all_df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = all_df.select_dtypes(exclude=[np.number]).columns.tolist()

In [44]:
print(cat_cols)

['ID', 'Segment', '최종카드론_신청경로코드']


In [47]:
del cat_cols[0]

In [49]:
print(cat_cols)

['Segment', '최종카드론_신청경로코드']


In [51]:
print(num_cols)

['기준년월', 'RP건수_B0M', 'RP금액_B0M', 'RP유형건수_B0M', 'RP건수_통신_B0M', 'RP건수_아파트_B0M', 'RP건수_제휴사서비스직접판매_B0M', 'RP건수_렌탈_B0M', 'RP건수_가스_B0M', 'RP건수_전기_B0M', 'RP건수_보험_B0M', 'RP건수_학습비_B0M', 'RP건수_교통_B0M', 'RP후경과월', 'RP후경과월_통신', 'RP후경과월_아파트', 'RP후경과월_제휴사서비스직접판매', 'RP후경과월_렌탈', 'RP후경과월_가스', 'RP후경과월_전기', 'RP후경과월_보험', 'RP후경과월_학습비', 'RP후경과월_교통', '증감_RP건수_전월', '증감_RP유형건수_전월', '증감_RP건수_통신_전월', '증감_RP건수_아파트_전월', '증감_RP건수_제휴사서비스직접판매_전월', '증감_RP건수_렌탈_전월', '증감_RP건수_가스_전월', '증감_RP건수_전기_전월', '증감_RP건수_보험_전월', '증감_RP건수_학습비_전월', '증감_RP건수_교통_전월', '최초카드론이용경과월', '최종카드론이용경과월', '카드론이용건수_누적', '카드론이용월수_누적', '카드론이용금액_누적', '최종카드론_금융상환방식코드', '최종카드론_대출이율', '최종카드론_대출일자', '최종카드론_대출월수', '최종카드론_거치개월수', '최종카드론_대출금액', '이용개월수_온라인_R6M', '이용개월수_오프라인_R6M', '이용금액_온라인_R6M', '이용금액_오프라인_R6M', '이용건수_온라인_R6M', '이용건수_오프라인_R6M', '이용금액_온라인_R3M', '이용금액_오프라인_R3M', '이용건수_온라인_R3M', '이용건수_오프라인_R3M', '이용금액_온라인_B0M', '이용금액_오프라인_B0M', '이용건수_온라인_B0M', '이용건수_오프라인_B0M', '이용개월수_페이_온라인_R6M', '이용개월수_페이_오프라인_R6M', '이용금액_페이_온라인_R6M', '이용금액_페이_오프라인_R6M', '이용건수_

In [53]:
del num_cols[0]

In [55]:
print(num_cols)

['RP건수_B0M', 'RP금액_B0M', 'RP유형건수_B0M', 'RP건수_통신_B0M', 'RP건수_아파트_B0M', 'RP건수_제휴사서비스직접판매_B0M', 'RP건수_렌탈_B0M', 'RP건수_가스_B0M', 'RP건수_전기_B0M', 'RP건수_보험_B0M', 'RP건수_학습비_B0M', 'RP건수_교통_B0M', 'RP후경과월', 'RP후경과월_통신', 'RP후경과월_아파트', 'RP후경과월_제휴사서비스직접판매', 'RP후경과월_렌탈', 'RP후경과월_가스', 'RP후경과월_전기', 'RP후경과월_보험', 'RP후경과월_학습비', 'RP후경과월_교통', '증감_RP건수_전월', '증감_RP유형건수_전월', '증감_RP건수_통신_전월', '증감_RP건수_아파트_전월', '증감_RP건수_제휴사서비스직접판매_전월', '증감_RP건수_렌탈_전월', '증감_RP건수_가스_전월', '증감_RP건수_전기_전월', '증감_RP건수_보험_전월', '증감_RP건수_학습비_전월', '증감_RP건수_교통_전월', '최초카드론이용경과월', '최종카드론이용경과월', '카드론이용건수_누적', '카드론이용월수_누적', '카드론이용금액_누적', '최종카드론_금융상환방식코드', '최종카드론_대출이율', '최종카드론_대출일자', '최종카드론_대출월수', '최종카드론_거치개월수', '최종카드론_대출금액', '이용개월수_온라인_R6M', '이용개월수_오프라인_R6M', '이용금액_온라인_R6M', '이용금액_오프라인_R6M', '이용건수_온라인_R6M', '이용건수_오프라인_R6M', '이용금액_온라인_R3M', '이용금액_오프라인_R3M', '이용건수_온라인_R3M', '이용건수_오프라인_R3M', '이용금액_온라인_B0M', '이용금액_오프라인_B0M', '이용건수_온라인_B0M', '이용건수_오프라인_B0M', '이용개월수_페이_온라인_R6M', '이용개월수_페이_오프라인_R6M', '이용금액_페이_온라인_R6M', '이용금액_페이_오프라인_R6M', '이용건수_페이_온라인_R

In [61]:
anova = []
chi = []

for col in all_df.columns:
    if col == 'Segment':
        continue

    if col in num_cols:
        data = all_df[[col, 'Segment']].dropna()
        groups = [data[data['Segment'] == val][col] for val in data['Segment'].unique()]
        try:
            stat = f_oneway(*groups).statistic
            ss_between = sum([(g.mean() - data[col].mean())**2 * len(g) for g in groups])
            ss_total = sum((data[col] - data[col].mean())**2)
            eta2 = eta_squared(ss_between, ss_total)
            anova.append({'변수': col, '유형': '수치형', '계수종류': 'Eta²', '상관계수': eta2})
        except:
            continue

    elif col in cat_cols:
        contingency = pd.crosstab(all_df[col], all_df['Segment'])
        if contingency.shape[0] > 1 and contingency.shape[1] > 1:
            try:
                v = cramers_v(contingency)
                chi.append({'변수': col, '유형': '범주형', '계수종류': "Cramér's V", '상관계수': v})
            except:
                continue

# 결과 정리
result_df1 = pd.DataFrame(anova)
result_df2 = pd.DataFrame(chi)
result_df1 = result_df1.sort_values(by='상관계수', ascending=False).reset_index(drop=True)
result_df2 = result_df2.sort_values(by='상관계수', ascending=False).reset_index(drop=True)

# 결과 출력
display(result_df1)
display(result_df2)

,변수,유형,계수종류,상관계수
0,이용금액_오프라인_B0M,수치형,Eta²,0.328568
1,이용금액_오프라인_R3M,수치형,Eta²,0.293962
2,이용금액_오프라인_R6M,수치형,Eta²,0.291066
3,이용건수_오프라인_B0M,수치형,Eta²,0.215746
4,이용건수_오프라인_R6M,수치형,Eta²,0.205766
...,...,...,...,...
73,증감_RP건수_렌탈_전월,수치형,Eta²,0.000012
74,증감_RP건수_보험_전월,수치형,Eta²,0.000006
75,증감_RP건수_통신_전월,수치형,Eta²,0.000005
76,증감_RP건수_가스_전월,수치형,Eta²,0.000004


,변수,유형,계수종류,상관계수
0,최종카드론_신청경로코드,범주형,Cramér's V,0.072631


In [62]:
pd.set_option('display.max_rows', 100)
display(result_df1)

,변수,유형,계수종류,상관계수
0,이용금액_오프라인_B0M,수치형,Eta²,0.328568
1,이용금액_오프라인_R3M,수치형,Eta²,0.293962
2,이용금액_오프라인_R6M,수치형,Eta²,0.291066
3,이용건수_오프라인_B0M,수치형,Eta²,0.215746
4,이용건수_오프라인_R6M,수치형,Eta²,0.205766
5,이용건수_오프라인_R3M,수치형,Eta²,0.205711
6,이용금액_온라인_B0M,수치형,Eta²,0.152713
7,이용금액_페이_온라인_B0M,수치형,Eta²,0.145210
8,이용건수_온라인_R6M,수치형,Eta²,0.137876
9,이용건수_온라인_R3M,수치형,Eta²,0.134742


In [63]:
result_df1['상관계수'] = result_df1['상관계수'].round(4)

In [64]:
display(result_df1)

,변수,유형,계수종류,상관계수
0,이용금액_오프라인_B0M,수치형,Eta²,0.3286
1,이용금액_오프라인_R3M,수치형,Eta²,0.2940
2,이용금액_오프라인_R6M,수치형,Eta²,0.2911
3,이용건수_오프라인_B0M,수치형,Eta²,0.2157
4,이용건수_오프라인_R6M,수치형,Eta²,0.2058
5,이용건수_오프라인_R3M,수치형,Eta²,0.2057
6,이용금액_온라인_B0M,수치형,Eta²,0.1527
7,이용금액_페이_온라인_B0M,수치형,Eta²,0.1452
8,이용건수_온라인_R6M,수치형,Eta²,0.1379
9,이용건수_온라인_R3M,수치형,Eta²,0.1347


### corr 상관계수

In [65]:
# all_df에서 기준년월, ID 컬럼은 제거한다. 
all_df = all_df.drop(['기준년월','ID'], axis=1)

In [66]:
cat_cols

['Segment', '최종카드론_신청경로코드']

In [67]:
# A가 가장 높은 등급일 것이라 추측하고 세그먼트만 LabelEncoder가 아닌 map으로 진행

segment_order = {'E': 0, 'D': 1, 'C': 2, 'B': 3, 'A': 4}
all_df['Segment_e'] = all_df['Segment'].map(segment_order)

In [79]:
# 인코딩 진행
# 인코딩할 컬럼만 선택
encoder = LabelEncoder()
all_df['최종카드론_신청경로코드'] = encoder.fit_transform(all_df['최종카드론_신청경로코드'])

In [81]:
print(all_df['Segment_e'].unique())

[1 0 2 4 3]


In [85]:
all_df['최종카드론_신청경로코드'].value_counts()

최종카드론_신청경로코드
5    1958226
3     230642
2     131317
0      55107
4      16713
1       7995
Name: count, dtype: int64

In [83]:
recols = num_cols + ['최종카드론_신청경로코드']
print(recols)

['RP건수_B0M', 'RP금액_B0M', 'RP유형건수_B0M', 'RP건수_통신_B0M', 'RP건수_아파트_B0M', 'RP건수_제휴사서비스직접판매_B0M', 'RP건수_렌탈_B0M', 'RP건수_가스_B0M', 'RP건수_전기_B0M', 'RP건수_보험_B0M', 'RP건수_학습비_B0M', 'RP건수_교통_B0M', 'RP후경과월', 'RP후경과월_통신', 'RP후경과월_아파트', 'RP후경과월_제휴사서비스직접판매', 'RP후경과월_렌탈', 'RP후경과월_가스', 'RP후경과월_전기', 'RP후경과월_보험', 'RP후경과월_학습비', 'RP후경과월_교통', '증감_RP건수_전월', '증감_RP유형건수_전월', '증감_RP건수_통신_전월', '증감_RP건수_아파트_전월', '증감_RP건수_제휴사서비스직접판매_전월', '증감_RP건수_렌탈_전월', '증감_RP건수_가스_전월', '증감_RP건수_전기_전월', '증감_RP건수_보험_전월', '증감_RP건수_학습비_전월', '증감_RP건수_교통_전월', '최초카드론이용경과월', '최종카드론이용경과월', '카드론이용건수_누적', '카드론이용월수_누적', '카드론이용금액_누적', '최종카드론_금융상환방식코드', '최종카드론_대출이율', '최종카드론_대출일자', '최종카드론_대출월수', '최종카드론_거치개월수', '최종카드론_대출금액', '이용개월수_온라인_R6M', '이용개월수_오프라인_R6M', '이용금액_온라인_R6M', '이용금액_오프라인_R6M', '이용건수_온라인_R6M', '이용건수_오프라인_R6M', '이용금액_온라인_R3M', '이용금액_오프라인_R3M', '이용건수_온라인_R3M', '이용건수_오프라인_R3M', '이용금액_온라인_B0M', '이용금액_오프라인_B0M', '이용건수_온라인_B0M', '이용건수_오프라인_B0M', '이용개월수_페이_온라인_R6M', '이용개월수_페이_오프라인_R6M', '이용금액_페이_온라인_R6M', '이용금액_페이_오프라인_R6M', '이용건수_페이_온라인_R

In [87]:
# 비교할 종속변수
target = 'Segment_e'
# 결과를 담을 리스트 
corr_result = []
# 반복
for col in recols:
    if col in all_df.columns:
        corr = all_df[col].corr(all_df[target])
        corr_result.append({'변수': col, '상관계수': corr})

# 데이터프레임으로 정리한다 
result_df3 = pd.DataFrame(corr_result).sort_values(by='상관계수', key=abs, ascending=False).reset_index(drop=True)

display(result_df3)

,변수,상관계수
0,이용금액_오프라인_B0M,0.566186
1,이용금액_오프라인_R3M,0.539827
2,이용금액_오프라인_R6M,0.538775
3,이용건수_오프라인_B0M,0.458959
4,이용건수_오프라인_R6M,0.450232
5,이용건수_오프라인_R3M,0.449336
6,이용금액_온라인_B0M,0.389504
7,이용금액_페이_온라인_B0M,0.378887
8,이용건수_온라인_R6M,0.367225
9,이용금액_페이_온라인_R3M,0.364122


In [89]:
result_AK = pd.concat(
    [result_df1, result_df2],
    axis=0,
    ignore_index=True
)
display(result_AK)

,변수,유형,계수종류,상관계수
0,이용금액_오프라인_B0M,수치형,Eta²,0.328600
1,이용금액_오프라인_R3M,수치형,Eta²,0.294000
2,이용금액_오프라인_R6M,수치형,Eta²,0.291100
3,이용건수_오프라인_B0M,수치형,Eta²,0.215700
4,이용건수_오프라인_R6M,수치형,Eta²,0.205800
5,이용건수_오프라인_R3M,수치형,Eta²,0.205700
6,이용금액_온라인_B0M,수치형,Eta²,0.152700
7,이용금액_페이_온라인_B0M,수치형,Eta²,0.145200
8,이용건수_온라인_R6M,수치형,Eta²,0.137900
9,이용건수_온라인_R3M,수치형,Eta²,0.134700


In [91]:
### 내보낸다. 
result_AK.to_excel('승인매출정보3-3_상관계수.xlsx', index=False)
result_df3.to_excel('승인매출정보3-3_검사.xlsx', index=False)

### VIF

In [23]:
# 사용할 수치형 변수 리스트
features = ['최초한도금액', '카드이용한도금액',
       'CA한도금액', 'CA이자율_할인전', '카드이용한도금액_B1M',
       '카드이용한도금액_B2M']
features1 = ['최초한도금액',
       'CA이자율_할인전',
       'CA한도금액']
features2 = ['최초한도금액', '카드이용한도금액',
       'CA한도금액', 'CA이자율_할인전', '카드이용한도금액_B2M']
# 독립변수 데이터
X = all_df[features2]
X = add_constant(X)  # 상수항 추가

# VIF 계산
vif_df = pd.DataFrame()
vif_df['Feature'] = X.columns
vif_df['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_df)

        Feature        VIF
0         const  54.048339
1        최초한도금액   1.019199
2      카드이용한도금액  19.091935
3        CA한도금액   7.152437
4     CA이자율_할인전   1.064573
5  카드이용한도금액_B2M  16.313782


In [24]:
all_df[column5].corr()

,한도증액횟수_R12M,한도증액금액_R12M,한도증액후경과월,상향가능한도금액,상향가능CA한도금액,Segment_e
한도증액횟수_R12M,1.000000,0.941650,-0.819507,0.032296,0.034784,0.060724
한도증액금액_R12M,0.941650,1.000000,-0.763369,0.036540,0.040480,0.084180
한도증액후경과월,-0.819507,-0.763369,1.000000,0.003790,0.001530,-0.039390
상향가능한도금액,0.032296,0.036540,0.003790,1.000000,0.963278,0.050764
상향가능CA한도금액,0.034784,0.040480,0.001530,0.963278,1.000000,0.064575
Segment_e,0.060724,0.084180,-0.039390,0.050764,0.064575,1.000000


### CramersV

In [26]:
# 범주형 vs 범주형: Cramér's V
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    if confusion_matrix.shape[0] < 2 or confusion_matrix.shape[1] < 2:
        return np.nan  # 최소 2x2 이상 필요
    try:
        chi2 = chi2_contingency(confusion_matrix)[0]
        n = confusion_matrix.sum().sum()
        phi2 = chi2 / n
        r, k = confusion_matrix.shape
        return np.sqrt(phi2 / min(k - 1, r - 1))
    except:
        return np.nan


In [27]:
combined_categorical_cols = [
    '자발한도감액횟수_R12M',
    '자발한도감액금액_R12M',
    '자발한도감액후경과월',
    '강제한도감액횟수_R12M',
    '강제한도감액금액_R12M',
    '강제한도감액후경과월',
    '카드론동의여부',
    '월상환론상향가능한도금액',
    'RV전환가능여부',
    '일시불ONLY전환가능여부',
    '특별한도보유여부_R3M',
    '연체감액여부_R3M',
    '한도증액횟수_R12M',
    '한도증액금액_R12M',
    '한도증액후경과월',
    '상향가능한도금액',
    '상향가능CA한도금액',
    '한도심사요청건수',
    '한도요청거절건수',
    '한도심사요청후경과월',
    '한도심사거절후경과월',
    '시장단기연체여부_R6M',
    '시장단기연체여부_R3M',
    '시장연체상환여부_R6M',
    'rv최초시작후경과일'
]


numerical_cols = ['최초한도금액',
 '카드이용한도금액',
 'CA한도금액',
 '일시상환론한도금액',
 '월상환론한도금액',
 'CA이자율_할인전',
 'CL이자율_할인전','RV일시불이자율_할인전',
 'RV현금서비스이자율_할인전',
 'RV약정청구율',
 'RV최소결제비율',
 '카드이용한도금액_B1M',
 '카드이용한도금액_B2M']

In [28]:
# 변수 정의
segment_col = 'Segment'
categorical_cols = combined_categorical_cols  # Segment_e 제외된 범주형 컬럼 리스트

results = []

for col in categorical_cols:
    if col == segment_col:
        continue

    coeff = cramers_v(all_df[segment_col], all_df[col])
    coef_type = "Cramér's V"
    var_type = "범주형"

    results.append({
        '변수': col,
        '유형': var_type,
        '계수종류': coef_type,
        '상관계수': round(coeff, 6) if not pd.isna(coeff) else np.nan
    })

correlation_df = pd.DataFrame(results).sort_values(by='상관계수', ascending=False)
display(correlation_df)



,변수,유형,계수종류,상관계수
15,상향가능한도금액,범주형,Cramér's V,0.141612
16,상향가능CA한도금액,범주형,Cramér's V,0.136274
13,한도증액금액_R12M,범주형,Cramér's V,0.078676
24,rv최초시작후경과일,범주형,Cramér's V,0.070303
12,한도증액횟수_R12M,범주형,Cramér's V,0.070096
8,RV전환가능여부,범주형,Cramér's V,0.044128
6,카드론동의여부,범주형,Cramér's V,0.038021
5,강제한도감액후경과월,범주형,Cramér's V,0.029208
4,강제한도감액금액_R12M,범주형,Cramér's V,0.028309
14,한도증액후경과월,범주형,Cramér's V,0.027772


모든 조합이 True라면 Segment마다 평균이 모두 달라 매우 유의미한 변수